In [1]:
# !pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers
# !pip install python-dotenv

# load variable from .env 
from dotenv import load_dotenv
load_dotenv()
import os


In [2]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)
hf_auth  = os.getenv("HF_AUTH_TOKEN")

# begin initializing HF items, you need an access token
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)

# enable evaluation mode to allow model inference
model.eval()

print(f"Model loaded on {device}")

/home/nghiaph/GEC/GEC_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nghiaph/GEC/GEC_env/lib/python3.8/site-packages/transformers/models/auto/configuration_auto.py:1006: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/nghiaph/GEC/GEC_env/lib/python3.8/site-packages/transformers/models/auto/auto_factory.py:479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.13s/it]
/home/nghiaph/GEC/GEC_env/lib/python3.8/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Model loaded on cuda:0


In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/home/nghiaph/GEC/GEC_env/lib/python3.8/site-packages/transformers/models/auto/tokenization_auto.py:640: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [4]:
stop_list = ['\nHuman:', '\n```\n']

stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids

[[1, 29871, 13, 29950, 7889, 29901], [1, 29871, 13, 28956, 13]]

In [5]:
import torch

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids


[tensor([    1, 29871,    13, 29950,  7889, 29901], device='cuda:0'),
 tensor([    1, 29871,    13, 28956,    13], device='cuda:0')]

In [6]:
from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [7]:
generate_text = transformers.pipeline(
    model=model, 
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [8]:
# res = generate_text("Explain me the difference between Data Lakehouse and Data Warehouse.")
# print(res[0]["generated_text"])

# Implementing HF Pipeline in LangChain

In [9]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

# checking again that everything is working fine
text = llm(prompt="How to succeed in entrepreneurship?")
print(text)



 nobody knows the answer to that question better than Elon Musk. As the CEO of SpaceX and Tesla, he has built two of the most successful and innovative companies of our time. And yet, despite his incredible success, he still faces challenges and setbacks on a daily basis. In this article, we will explore some of the key lessons that Musk has learned throughout his career, and how they can help you succeed in entrepreneurship.
1. Think Big: One of the most important lessons that Musk has learned is the importance of thinking big. He believes that the only way to truly make an impact is to aim high and push the boundaries of what is possible. "I think it's important to have a vision for the future that is much more ambitious than what we have today," Musk says. "We need to be thinking about how to solve some of the world's biggest problems, like climate change and sustainable energy."
2. Take Risks: Another key lesson that Musk has learned is the importance of taking risks. He believes 

In [10]:
from langchain.document_loaders import WebBaseLoader

web_links = ["https://www.databricks.com/","https://help.databricks.com","https://databricks.com/try-databricks","https://help.databricks.com/s/","https://docs.databricks.com","https://kb.databricks.com/","http://docs.databricks.com/getting-started/index.html","http://docs.databricks.com/introduction/index.html","http://docs.databricks.com/getting-started/tutorials/index.html","http://docs.databricks.com/release-notes/index.html","http://docs.databricks.com/ingestion/index.html","http://docs.databricks.com/exploratory-data-analysis/index.html","http://docs.databricks.com/data-preparation/index.html","http://docs.databricks.com/data-sharing/index.html","http://docs.databricks.com/marketplace/index.html","http://docs.databricks.com/workspace-index.html","http://docs.databricks.com/machine-learning/index.html","http://docs.databricks.com/sql/index.html","http://docs.databricks.com/delta/index.html","http://docs.databricks.com/dev-tools/index.html","http://docs.databricks.com/integrations/index.html","http://docs.databricks.com/administration-guide/index.html","http://docs.databricks.com/security/index.html","http://docs.databricks.com/data-governance/index.html","http://docs.databricks.com/lakehouse-architecture/index.html","http://docs.databricks.com/reference/api.html","http://docs.databricks.com/resources/index.html","http://docs.databricks.com/whats-coming.html","http://docs.databricks.com/archive/index.html","http://docs.databricks.com/lakehouse/index.html","http://docs.databricks.com/getting-started/quick-start.html","http://docs.databricks.com/getting-started/etl-quick-start.html","http://docs.databricks.com/getting-started/lakehouse-e2e.html","http://docs.databricks.com/getting-started/free-training.html","http://docs.databricks.com/sql/language-manual/index.html","http://docs.databricks.com/error-messages/index.html","http://www.apache.org/","https://databricks.com/privacy-policy","https://databricks.com/terms-of-use"] 

loader = WebBaseLoader(web_links)
documents = loader.load()

In [11]:
# describe the documents
print('type of documents:', type(documents))
print('shape of documents:', len(documents))
doc_example = documents[0]
print('doc_example :', doc_example.page_content)
print('type of doc_example:', type(doc_example))

type of documents: <class 'list'>
shape of documents: 39
doc_example : Data Lakehouse Architecture and AI Company | DatabricksSkip to main contentWhy DatabricksWhy Choose Databricks?For executivesFor startups CustomersFeaturedSee all customersPartnersCloud ProvidersTechnology & Data PartnersTechnology PartnersConnect your existing tools to your LakehouseData PartnersAccess the ecosystem of data consumersBuilt on DatabricksBuild, market and grow your businessConsulting & System IntegratorsC&SI Partner ProgramBuild, deploy or migrate to the LakehousePartner SolutionsFind custom industry and migration solutionsSecurity & Trust CenterThe Data LakehouseData + AI in the real worldDiscover the latest trends in data and AI adoption across 9000+ organizationsGet the eBookProductLakehouse PlatformLakehouse Platform OverviewA unified platform for data, analytics and AIData SharingAn open, secure, zero-copy sharing for all dataData GovernanceUnified governance for all data, analytics and AI assets

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings( model_name=model_name, model_kwargs=model_kwargs)

# storing embeddings in the vector store
vectorstore = FAISS.from_documents(all_splits, embeddings)

## Initializing Chain

In [14]:
from langchain.chains import ConversationalRetrievalChain

chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

In [15]:
chat_history = []

query = "What is Data lakehouse architecture in Databricks?"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])

 In Databricks, the medallion lakehouse architecture refers to a multi-layered approach to validating, cleaning, and transforming data for analytics. This architecture includes the following layers:























































































































































































































































































































































































































































































In [16]:
print(result['source_documents'])

[Document(page_content='What is data modeling on Databricks? \nThe Databricks Lakehouse Platform organizes data stored with Delta Lake in cloud object storage with familiar relations like database schemas, tables, and views. Databricks recommends a multi-layer approach to validating, cleansing, and transforming data for analytics. For more information, see the medallion architecture.\n\n\nWhat is Databricks SQL? \nDatabricks SQL provides general compute resources for SQL queries, visualizations, and dashboards that are executed against the tables in the lakehouse. Within Databricks SQL, these queries, visualizations, and dashboards are developed and executed using SQL editor.', metadata={'source': 'http://docs.databricks.com/sql/index.html', 'title': 'What is data warehousing on Databricks? | Databricks on AWS', 'description': 'Learn about building a data warehousing solution in Databricks using Databricks SQL.', 'language': 'en-US'}), Document(page_content='Data governance\nLakehouse 

In [17]:
chat_history = [(query, result["answer"])]

query = "What are Data Governance and Interoperability in it?"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])
chat_history.append((query, result["answer"]))

  No, I don't know the answer to this question as it doesn't relate to any piece of context provided in the prompt.


In [22]:
query = "what is databricks what solutions does it provide?"
result = chain({"question": query, "chat_history": chat_history})

In [23]:
print(result['answer'])

  There is no one "best" way to learn MS SQL, as different people prefer different methods and resources. However, here are some popular ways to learn MS SQL:































































































































































































































































































































































































































































































In [20]:
print(result['source_documents'])


[Document(page_content='COPY INTO\nDELETE FROM\nINSERT INTO\nINSERT OVERWRITE DIRECTORY\nINSERT OVERWRITE DIRECTORY with Hive format\nLOAD DATA\nMERGE INTO\nUPDATE\n\n\n\n\nData retrieval statements \nYou use a query to retrieve rows from one or more tables according to the specified clauses. The full syntax\nand brief description of supported clauses are explained in the Query article.\nThe related SQL statements SELECT and VALUES are also included in this section.\n\n\nQuery\nSELECT\nVALUES\n\n\nDatabricks SQL also provides the ability to generate the logical and physical plan for a query using the EXPLAIN statement.\n\n\nEXPLAIN\n\n\n\n\nDelta Lake statements \nYou use Delta Lake SQL statements to manage tables stored in Delta Lake format:\n\n\nCACHE SELECT\nCONVERT TO DELTA\nDESCRIBE HISTORY\nFSCK REPAIR TABLE\nGENERATE\nOPTIMIZE\nREORG TABLE\nRESTORE\nVACUUM\n\n\n\nFor details on using Delta Lake statements, see What is Delta Lake?.', metadata={'source': 'http://docs.databricks.co

In [21]:
print(result)

{'question': 'what is databricks what solutions does it provide?', 'chat_history': [('What is Data lakehouse architecture in Databricks?', ' In Databricks, the medallion lakehouse architecture refers to a multi-layered approach to validating, cleaning, and transforming data for analytics. This architecture includes the following layers:\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n